## Data Processing - ECS171 Project Group 8

**Description**: Converts true/false entries to 1/0 and normalizes numerical data

In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [18]:
path_nontrend = 'datasets/11.09 nontrending.csv'
path_trend = 'datasets/11.09 trending.csv'
df_nontrend = pd.read_csv(path_nontrend)
df_trend = pd.read_csv(path_trend)
df_nontrend.head()

df_nontrend = df_nontrend.drop(['dimension'],axis=1)
df_trend = df_trend.drop(['dimension'],axis=1)
pd.set_option('display.max_columns', None)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (18,19,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
# def handle_duplicates(dfn, dft):
    
#     trend_later_list = []
#     drop_list = []
    
#     for row in dfn.index:
#         if dfn['video_id'][row] in dft['video_id']:
#             trend_later_list.append(dfn['video_id'][row])
#             drop_list.append(row)
    
#     dft['trend_later'] = False
#     for ids in trend_later_list:
#         dft.iloc[dft[dft['video_id'] == ids].index.values]['trend_later'] = True
#     dfn.drop(drop_list, axis = 0, inplace = True)
#     dfn['trend_later'] = False
    
def excess_remove(dft, dfn):
    excess = []
    for col in dfn.columns:
        if col not in dft.columns:
            excess.append(col)
    dfn.drop(excess, axis = 1, inplace = True)
    
def update_remove(df, kept_date):
    prospects = []
    remove = []
    for cols in df.columns:
        if "timestamp" in cols or "likes" in cols or "dislikes" in cols or "comment" in cols or "view" in cols:
            prospects.append(cols)
    for col2 in prospects:
        if kept_date not in col2 and  "Channel_viewCount" not in col2 :
            remove.append(col2)
    return remove

#convert quotations '\"'
#function: remove_quote(column)
#use on nontrend (definition, Channel_country) and trend (Channel_country)
def remove_quote(col):
    lst = []
    for v in col:
        if type(v) == str:
            lst.append(v.replace('"',''))
        else:
            lst.append(v)
    return lst


#sets 'tags' to number of tags
#function: count_tags(dataframe)
#use on nontrend and trend
def count_tags(df):
    lst=[]
    for entry in df['tags']:
        if entry == '[none]':
            lst.append(0)
        else:
            num_tags = entry.count('|')
            lst.append(num_tags)
    df['tags'] = lst


#Channel_country: 0 for INTL, 1 for USA
#function: encode_country(dataframe)
#use on nontrend and trend
def encode_country(df):
    lst = []
    for entry in df['Channel_country']:
        if entry == 'US':
            lst.append("USA")
        elif entry == "":
            lst.append("UNK")
        else:
            lst.append("INTL")
    df['Channel_country'] = lst

#remove Channel_hiddenSubscriberCount == True rows
#function: clean_subcount(dataframe)
#use on nontrend
def clean_subcount(df):
    lst = []
    for i in df.index:
        if df['Channel_hiddenSubscriberCount'][i] == True:
            lst.append(i)
    return lst

In [21]:
from os import listdir
from os.path import isfile, join
import pandas as pd
#if a video is in both '11.09 nontrending' and '11.09 trending', drop it from nontrending (it was never nontrending.) If it shows up in one of the later trending video sets, add that information

#pass trending dframe as dft, nontrending as dfn
def handle_trending(dft,dfn):
    path= './datasets'
    trending = [f for f in listdir(path) if isfile(join(path, f)) and " trending" in f]

    all_trending_ids = set()
    for i in trending:
        all_trending_ids.update(pd.read_csv(f"{path}/{i}")["video_id"].to_list())

    original_trending_ids = dft["video_id"].to_list()

    trended = set()
    duplicate = set()
    for i in dfn.index:
        if dfn["video_id"][i] in all_trending_ids:
            trended.update([i])
        if dfn["video_id"][i] in original_trending_ids:
            duplicate.update([i])
    trended = trended.symmetric_difference(duplicate)

    #drop videos which are duplicates of videos in the "11.09 trending" database
    dfn.drop(duplicate, inplace=True)

    #these videos were not originally trending, but started trending later. 
    dfn.insert(10,"trended_later",False)
    dfn.loc[trended,"trended_later"]=True

In [22]:
handle_trending(df_trend, df_nontrend)

In [23]:
df_trend['trending?'] = True
df_nontrend['trending?'] = False

In [24]:
excess_remove(df_trend, df_nontrend)
frames = [df_trend, df_nontrend]
# Link that teaches how to concatenate:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
df = pd.concat(frames, ignore_index=True, sort=False)
#df.reset_index(drop=True)
#df = df_trend
df = df.drop(['Unnamed: 0','is_trending'], axis=1)
#df.to_csv("~/CS/ECS-171-Group8/orig_DataProcessNotebook.csv")
#df.reset_index(drop=True)
df.dropna(inplace= True)

In [25]:
# df[df['duration'] != type(str)]

index = 0
lst_drop = []
for i in df['duration']:
    if type(i) != str:
        lst_drop.append(index)
    index += 1
df.drop(lst_drop, inplace = True)   

In [26]:
#-----------------------------
# installation: pip3 install isodate
import isodate
duration_seconds = []

for i in df['duration']:
    dur = isodate.parse_duration(i.replace('"','' ))
    duration_seconds.append(dur.total_seconds())
df['duration'] = duration_seconds

In [27]:
def engagementRate(likes, comment, subscriber):
    return (likes + comment)/subscriber

#drop any na values from the dataset
df = df.dropna()

like_col = [col for col in df if col.startswith('likes') ]
comment_col = [col for col in df if col.startswith('comment') ]
subscriber_col = [col for col in df if col.startswith('Channel_subscriberCount') ]

#feed the data to the formula and store the result in engagement rate
df["engagement_rate"] = engagementRate(df[like_col[len(like_col)-1]],df[comment_col[len(comment_col)-1]], df[subscriber_col[len(subscriber_col)-1]])

#replace any infinite values with nan
df["engagement_rate"] = df["engagement_rate"].replace([np.inf, -np.inf], np.nan)

In [28]:
new_df = df.drop(update_remove(df, "11_19"), axis = 1)

In [15]:
from sklearn.preprocessing import StandardScaler
numeric_cols = ['duration', 'Channel_viewCount', 'Channel_subscriberCount', 'Channel_videoCount', 'view_count_update_11_19_14', 'likes_update_11_19_14', 'dislikes_update_11_19_14', 'comment_count_update_11_19_14',]
new_df[numeric_cols] = StandardScaler().fit_transform(new_df[numeric_cols])

In [13]:
# #plt.hist(new_df['Channel_country'])
# plt.subplots(figsize=(71,71))
# sns.countplot(new_df['Channel_country'])

In [16]:
new_df['definition'] = remove_quote(new_df['definition'])
new_df['Channel_country'] = remove_quote(new_df['Channel_country'])

count_tags(new_df)

encode_country(new_df)
new_df[pd.get_dummies(new_df['Channel_country']).columns] = pd.get_dummies(new_df['Channel_country'])[:]

new_df = new_df.drop(clean_subcount(new_df))

AttributeError: 'int' object has no attribute 'count'

# Sources
1. https://stackoverflow.com/questions/40950791/remove-quotes-from-string-in-python
